In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [26]:
# 改进的中文字体设置
plt.rcParams["font.family"] = ["SimHei", "WenQuanYi Micro Hei", "Heiti TC", "sans-serif"]
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# 尝试查找可用的中文字体
def set_chinese_font():
    import matplotlib.font_manager as fm
    font_names = [f.name for f in fm.fontManager.ttflist]
    
    # 常用中文字体列表
    chinese_fonts = ['SimHei', 'WenQuanYi Micro Hei', 'Heiti TC', 'Microsoft YaHei']
    
    for font in chinese_fonts:
        if font in font_names:
            plt.rcParams["font.family"] = font
            print(f"找到中文字体: {font}")
            return True
    
    print("未找到中文字体，图表中的中文可能无法正常显示")
    return False

# 设置中文字体
set_chinese_font()

找到中文字体: SimHei


True

In [27]:
# 读取数据
file_path = 'data_fin.csv'
data = pd.read_csv(file_path, encoding='ISO-8859-1')

In [28]:
# 查看列名
column_names = data.columns
print(column_names)

Index(['Fe', 'Co', 'Mn', 'Al', 'Ni', 'B', 'Hf', 'Si', 'Cu', 'P', 'Mo', 'Cr',
       'La', 'Ti', 'Sn', 'V', 'Ga', 'Nb', 'Zr', 'C', 'Gd', 'Y', 'Ta', 'DO3',
       'BCC', 'FCC', 'HCP', 'Orthorhombic', 'hexagonal', 'Tetragonal',
       'Primitive Cubic', 'Ms', 'Hc'],
      dtype='object')


In [29]:
# 查看 Ms 和 Hc 列的统计信息
print("\n原始数据 Ms 和 Hc 统计信息:")
print(data[['Ms', 'Hc']].describe())

# 绘制原始数据分布
plt.figure(figsize=(15, 10))


原始数据 Ms 和 Hc 统计信息:
               Ms            Hc
count  247.000000    247.000000
mean   126.419873   1902.245883
std     52.531877   3942.859450
min      1.081700      0.770000
25%     84.400000     35.860625
50%    125.700000    334.225380
75%    168.744311   1850.176213
max    225.824317  26045.000000


<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

In [30]:
# 原始 Ms 分布
plt.subplot(2, 2, 1)
sns.histplot(data['Ms'], bins=30, kde=True)
plt.title("原始 Ms 分布")
plt.xlabel("Ms")
plt.ylabel("频次")

# 原始 Hc 分布
plt.subplot(2, 2, 2)
sns.histplot(data['Hc'], bins=30, kde=True)
plt.title("原始 Hc 分布")
plt.xlabel("Hc")
plt.ylabel("频次")

# 原始 Ms 箱线图
plt.subplot(2, 2, 3)
sns.boxplot(y=data['Ms'])
plt.title("原始 Ms 箱线图")
plt.ylabel("Ms")

# 原始 Hc 箱线图
plt.subplot(2, 2, 4)
sns.boxplot(y=data['Hc'])
plt.title("原始 Hc 箱线图")
plt.ylabel("Hc")

plt.tight_layout()
plt.savefig('Ms_Hc原始数据分布.png', dpi=300, bbox_inches='tight')
plt.close()

In [31]:
# 1️⃣ 先处理 Ms，使用 IQR 方法
Q1_Ms = data['Ms'].quantile(0.25)
Q3_Ms = data['Ms'].quantile(0.75)
IQR_Ms = Q3_Ms - Q1_Ms

lower_bound_Ms = Q1_Ms - 1.5 * IQR_Ms
upper_bound_Ms = Q3_Ms + 1.5 * IQR_Ms
print(f"\n【Ms IQR 计算结果】")
print(f"Q1: {Q1_Ms}, Q3: {Q3_Ms}, IQR: {IQR_Ms}")
print(f"异常值下界: {lower_bound_Ms}, 上界: {upper_bound_Ms}")

# 过滤 Ms 的异常值
data_cleaned = data[(data['Ms'] >= lower_bound_Ms) & (data['Ms'] <= upper_bound_Ms)]


【Ms IQR 计算结果】
Q1: 84.4, Q3: 168.7443108718765, IQR: 84.3443108718765
异常值下界: -42.11646630781475, 上界: 295.2607771796913


In [32]:
# 新增：Ms列除以30添加到新列
data_cleaned['Ms_div30'] = data_cleaned['Ms'] / 30
print("\n【Ms_div30列统计信息】")
print(data_cleaned['Ms_div30'].describe())


【Ms_div30列统计信息】
count    247.000000
mean       4.213996
std        1.751063
min        0.036057
25%        2.813333
50%        4.190000
75%        5.624810
max        7.527477
Name: Ms_div30, dtype: float64


In [33]:
# 2️⃣ 处理 Hc：进行对数转换
data_cleaned['Hc_log'] = np.log1p(data_cleaned['Hc'])  # log(Hc + 1)

print("\n【对数转换后的 Hc_log 统计信息】")
print(data_cleaned['Hc_log'].describe())


【对数转换后的 Hc_log 统计信息】
count    247.000000
mean       5.527189
std        2.417791
min        0.570980
25%        3.606885
50%        5.814803
75%        7.523519
max       10.167619
Name: Hc_log, dtype: float64


In [34]:
# 处理后 Ms 分布
plt.subplot(2, 2, 1)
sns.histplot(data_cleaned['Ms'], bins=30, kde=True)
plt.title("处理后 Ms 分布")
plt.xlabel("Ms")
plt.ylabel("频次")

# 处理后 Hc_log 分布
plt.subplot(2, 2, 2)
sns.histplot(data_cleaned['Hc_log'], bins=30, kde=True)
plt.title("对数转换后 Hc 分布")
plt.xlabel("Hc_log")
plt.ylabel("频次")

# 处理后 Ms 箱线图
plt.subplot(2, 2, 3)
sns.boxplot(y=data_cleaned['Ms'])
plt.title("处理后 Ms 箱线图")
plt.ylabel("Ms")

# 处理后 Hc_log 箱线图
plt.subplot(2, 2, 4)
sns.boxplot(y=data_cleaned['Hc_log'])
plt.title("对数转换后 Hc 箱线图")
plt.ylabel("Hc_log")

plt.tight_layout()
plt.savefig('Ms_Hc处理后数据分布.png', dpi=300, bbox_inches='tight')
plt.close()

print("\n数据可视化完成，已生成Ms_Hc原始数据分布.png和Ms_Hc处理后数据分布.png文件")
print(f"处理后的数据量: {data_cleaned.shape[0]} 行")  


数据可视化完成，已生成Ms_Hc原始数据分布.png和Ms_Hc处理后数据分布.png文件
处理后的数据量: 247 行


In [35]:
# 4️⃣ 保存数据
save_original_Hc = False  # 是否保存原始尺度的 Hc，False 表示保存对数转换后的数据

if save_original_Hc:
    data_cleaned['Hc'] = np.expm1(data_cleaned['Hc_log'])  # 反变换回 Hc
    data_cleaned.drop(columns=['Hc_log'], inplace=True)  # 删除 Hc_log

cleaned_file_path = 'cleaned_data_V1.csv'
data_cleaned.to_csv(cleaned_file_path, index=False, encoding='ISO-8859-1')

print(f"\n清洗后的数据已保存至 {cleaned_file_path}")
print(f"最终数据量: {data_cleaned.shape[0]} 行")


清洗后的数据已保存至 cleaned_data_V1.csv
最终数据量: 247 行
